https://xang1234.github.io/multi-label/

In [26]:
import numpy as np
import pandas as pd

from sklearn.preprocessing import MultiLabelBinarizer

# from sklearn.pipeline import Pipeline

from sklearn.feature_extraction import text
from sklearn.feature_extraction.text import TfidfVectorizer

from skmultilearn.adapt import MLkNN
from sklearn.model_selection import GridSearchCV

from sklearn import metrics

import pickle
import time

In [37]:
# Import data from csv
df = pd.read_csv('../datasets/cleaned/combined_text.csv')

# drop duplicates
df.drop_duplicates(subset=['combined_text'], keep='first', ignore_index=True, inplace=True)

# Create multi labels from category
category_list = list(df.category.unique())
df['labels'] = df['category'].map(lambda x: [category_list.index(x)])

df.drop(columns=['category', 'asin'], inplace=True)
df.rename(columns={'combined_text': 'text'}, inplace=True)

In [31]:
category_list

['appliances',
 'arts_crafts_and_sewing',
 'automotive',
 'baby',
 'beauty',
 'cell_phones_and_accessories',
 'clothing_shoes_and_jewelry',
 'electronics',
 'grocery_and_gourmet_food',
 'health_and_personal_care',
 'home_and_kitchen',
 'industrial_and_scientific',
 'musical_instruments',
 'office_products',
 'patio_lawn_and_garden',
 'pet_supplies',
 'software',
 'sports_and_outdoors',
 'tools_and_home_improvement',
 'toys_and_games',
 'video_games']

In [38]:
df.head(1)

,text,labels
0,I have a 9 year old Badger 1 that needs replac...,[0]


In [33]:
my_stop_words = []#extra words. todo: create a text file to load from
stop_words = text.ENGLISH_STOP_WORDS.union(my_stop_words)

vectorizer = TfidfVectorizer(
    ngram_range=(1,1), 
    stop_words=stop_words,
    analyzer='word',
    token_pattern=r'[a-zA-Z]+',
)

In [34]:
X = vectorizer.fit_transform(df['text'])

In [39]:
mlb = MultiLabelBinarizer()
y = mlb.fit_transform(df['labels'])

In [42]:
y[500:510]

array([[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])

In [43]:
mlb.classes_

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20])

In [44]:
%%time
parameters = {'k': range(1,3), 's': [0.5, 0.7, 1.0]}
score = 'f1_macro'

clf = GridSearchCV(MLkNN(), parameters, scoring=score)
clf.fit(X, y)

print('best parameters :', classifier.best_params_, 'best score: ',
      clf.best_score_)

C:\Users\mriva\.conda\envs\torch\lib\site-packages\sklearn\utils\validation.py:70: FutureWarning: Pass n_neighbors=1 as keyword args. From version 0.25 passing these as positional arguments will result in an error
  FutureWarning)


KeyboardInterrupt: 

In [8]:
y_hat = clf.predict(X)

In [18]:
clf.score(X, y)

0.6839148902959502

In [9]:
metrics.accuracy_score(y, y_hat)

0.6839148902959502

In [15]:
probs = clf.predict_proba(X)

In [16]:
probs[0]

array([0.31374618, 0.07596179, 0.13106712, 0.01746021, 0.00793469,
       0.00815209, 0.00181063, 0.0152032 , 0.00208402, 0.21833584,
       0.00762823, 0.00257229, 0.00303513, 0.00821017, 0.0142055 ,
       0.00572368, 0.00683542, 0.04472159, 0.0508844 , 0.05894804,
       0.00547978])

In [10]:
inv_category_dict = {v: k for k, v in category_dict.items()}

count = 0
for coef in clf.coef_:
    words = list(zip(coef, vectorizer.get_feature_names()))
    top_words = sorted(words, reverse=True)[:40]
    bottom_words = sorted(words, reverse=False)[:40]
    
    print(f"===Category: {inv_category_dict[count]}===")
    print(f"-Top-")
    print([x[1] for x in top_words])
    print(f"-Bottom-")
    print([x[1] for x in bottom_words])
    print('')
    count+=1

===Category: appliances===
-Top-
['whirlpool', 'maytag', 'frigidaire', 'ge', 'amana', 'kenmore', 'chimney', 'jenn', 'washer', 'keg', 'hood', 'kegs', 'refrigerator', 'ice', 'roper', 'fridge', 'hotpoint', 'dryer', 'ductless', 'cooktop', 'element', 'range', 'broan', 'duct', 'ja', 'humidifier', 'cycle', 'kitchenaid', 'freezer', 'vent', 'burners', 'ducted', 'disposal', 'wr', 'kegerator', 'da', 'knobs', 'ventless', 'filter', 'ww']
-Bottom-
['phone', 'camera', 'lens', 'safe', 'battery', 'seat', 'car', 'coffee', 'case', 'mm', 'hair', 'bed', 'charger', 'bottle', 'bike', 'table', 'windows', 'galaxy', 'speakers', 'laptop', 'mattress', 'chair', 'card', 'cable', 'pool', 'speaker', 'computer', 'keyboard', 'honda', 'drive', 'bowl', 'iphone', 'oil', 'wear', 'vacuum', 'microwave', 'cream', 'play', 'tablet', 'engine']

===Category: arts_crafts_and_sewing===
-Top-
['singer', 'janome', 'bobbin', 'mannequin', 'bernina', 'yarn', 'brother', 'airbrush', 'serger', 'juki', 'stitch', 'sizzix', 'bobbins', 'cricut

===Category: home_and_kitchen===
-Top-
['juicer', 'duvet', 'cuisinart', 'curtain', 'comforter', 'slicer', 'futon', 'kettle', 'cooker', 'toaster', 'roomba', 'vitamix', 'dyson', 'blender', 'induction', 'shams', 'headboard', 'pitcher', 'carafe', 'krups', 'infuser', 'dough', 'purifier', 'beater', 'topper', 'peeler', 'ironing', 'stools', 'pan', 'ksm', 'mug', 'frother', 'foodsaver', 'waring', 'ozone', 'mattress', 'eureka', 'bodum', 'miele', 'jvm']
-Bottom-
['camera', 'crib', 'bike', 'phone', 'lens', 'stroller', 'play', 'gate', 'bassinet', 'feeder', 'doll', 'toy', 'software', 'tire', 'headphones', 'gun', 'scope', 'tires', 'macbook', 'pool', 'iphone', 'antenna', 'radio', 'ram', 'guitar', 'chart', 'holster', 'roof', 'galaxy', 'ink', 'waist', 'tripod', 'honda', 'ear', 'mic', 'lamp', 'headset', 'music', 'nikon', 'ford']

===Category: industrial_and_scientific===
-Top-
['stethoscope', 'toothpaste', 'microscope', 'amscope', 'winch', 'stethoscopes', 'hydrometer', 'epoxy', 'oreck', 'littman', 'filame

===Category: video_games===
-Top-
['game', 'ds', 'vita', 'saitek', 'controller', 'multiplayer', 'wii', 'console', 'joystick', 'dlc', 'psn', 'steam', 'controllers', 'nyko', 'xbox', 'kinect', 'razer', 'dsi', 'mechanical', 'dance', 'ps', 'code', 'campaign', 'offline', 'yoke', 'edition', 'account', 'dpi', 'character', 'gamepad', 'mod', 'play', 'psp', 'headset', 'games', 'op', 'simulator', 'violence', 'steelseries', 'mods']
-Bottom-
['phone', 'fit', 'water', 'size', 'unit', 'printer', 'bag', 'camera', 'bottle', 'machine', 'hair', 'samsung', 'radio', 'lens', 'speaker', 'blade', 'canon', 'diameter', 'office', 'door', 'filter', 'product', 'bike', 'ipad', 'seat', 'handle', 'apple', 'car', 'record', 'bulb', 'pump', 'nikon', 'kit', 'oil', 'china', 'mattress', 'dvd', 'height', 'tall', 'lid']



In [19]:
filename = '../models/category_logistic_regression_ovr.sav'
pickle.dump(clf, open(filename, 'wb'))